In [3]:
!pip install --upgrade pymongo certifi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 25.8 MB/s eta 0:00:00


In [4]:


# Install pymongo for MongoDB connection


# Import necessary libraries
from pymongo import MongoClient
import pprint

# Replace with your MongoDB Atlas connection string
connection_string = "mongodb+srv://wfn7ad:<password>@cluster0practice.tldyf.mongodb.net/?retryWrites=true&w=majority&appName=Cluster0Practice"

# Connect to MongoDB Atlas
client = MongoClient(connection_string)

# Access the sample_mflix database and the movies collection
db = client['sample_mflix']
collection = db['movies']


Exercise 1. Basic Searching and Filtering

In [10]:
first_action_movie = collection.find_one({'genres': 'Action'})

First Action Movie: {'_id': ObjectId('573a1390f29313caabcd5293'), 'plot': "Young Pauline is left a lot of money when her wealthy uncle dies. However, her uncle's secretary has been named as her guardian until she marries, at which time she will officially take ...", 'genres': ['Action'], 'runtime': 199, 'cast': ['Pearl White', 'Crane Wilbur', 'Paul Panzer', 'Edward Josè'], 'num_mflix_comments': 0, 'poster': 'https://m.media-amazon.com/images/M/MV5BMzgxODk1Mzk2Ml5BMl5BanBnXkFtZTgwMDg0NzkwMjE@._V1_SY1000_SX677_AL_.jpg', 'title': 'The Perils of Pauline', 'fullplot': 'Young Pauline is left a lot of money when her wealthy uncle dies. However, her uncle\'s secretary has been named as her guardian until she marries, at which time she will officially take possession of her inheritance. Meanwhile, her "guardian" and his confederates constantly come up with schemes to get rid of Pauline so that he can get his hands on the money himself.', 'languages': ['English'], 'released': datetime.datetime(1

In [11]:
# 5 movies released after the year 2000 .
movies_after_2000 = collection.find({"year": {"$gt": 2000}}).limit(5)

In [12]:
# 5 movies where the IMDb rating is greater than 8.5.
high_rated_movies = collection.find({"imdb.rating": {"$gt": 8.5}}).limit(5)

In [13]:
# all movies where the genre contains both "Action" and "Adventure".
action_adventure_movies = collection.find({"genres": {"$all": ["Action", "Adventure"]}}).limit(5)

Exercise 2. Sorting results

In [14]:
# 5 movies where the genre is "Comedy" and sort them by IMDb rating in descending order .
sorted_comedy_movies = collection.find({"genres": "Comedy"}).sort("imdb.rating", -1).limit(5)

In [15]:
# 5 movies where the genre is "Drama" and sort them by release year in ascending order.
sorted_drama_movies = collection.find({"genres": "Drama"}).sort("year", 1).limit(5)

Exercise 3. Aggregation Pipelines

In [16]:
# aggregation pipeline that calculates the average IMDb rating for each genre (Return the top 5 genres).
avg_rating_by_genre = collection.aggregate([
    {"$unwind": "$genres"},
    {"$group": {"_id": "$genres", "avg_rating": {"$avg": "$imdb.rating"}}},
    {"$sort": {"avg_rating": -1}},
    {"$limit": 5}
])

In [17]:
# 3.2 Write an aggregation pipeline to find the top 5 directors by the average IMDb rating of their movies.
top_directors = collection.aggregate([
    {"$group": {"_id": "$directors", "avg_rating": {"$avg": "$imdb.rating"}}},
    {"$sort": {"avg_rating": -1}},
    {"$limit": 5}
])

In [18]:
# 3.3 Write an aggregation pipeline to calculate the total number of movies released in each year (Sort the results by the year).
movies_per_year = collection.aggregate([
    {"$group": {"_id": "$year", "total_movies": {"$sum": 1}}},
    {"$sort": {"_id": 1}}
])

Exercise 4. Updated and Deleting Documents

In [19]:
# 4.1 Write a query to update the IMDb rating of a movie with the title "The Godfather" to 9.5.
collection.update_one({"title": "The Godfather"}, {"$set": {"imdb.rating": 9.5}})

UpdateResult({'n': 1, 'electionId': ObjectId('7fffffff00000000000000a6'), 'opTime': {'ts': Timestamp(1728337072, 32), 't': 166}, 'nModified': 1, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1728337072, 32), 'signature': {'hash': b'\x9cg\xd6Qz\xde\x7f$\x97q,\x1f\xfaN\xe5\xdfOH\xb95', 'keyId': 7376187726187462660}}, 'operationTime': Timestamp(1728337072, 32), 'updatedExisting': True}, acknowledged=True)

In [20]:
# 4.2 Write a query to update all movies where the genre is "Horror" and set their IMDb rating to 6.0 if it is currently null.
collection.update_many({"genres": "Horror", "imdb.rating": {"$exists": False}}, {"$set": {"imdb.rating": 6.0}})

UpdateResult({'n': 0, 'electionId': ObjectId('7fffffff00000000000000a6'), 'opTime': {'ts': Timestamp(1728337087, 29), 't': 166}, 'nModified': 0, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1728337087, 29), 'signature': {'hash': b'\xf6\xab\n\x82\x8b"V\x1f\xf9\xce\x93\xa7-@f\xf3w>\xf64', 'keyId': 7376187726187462660}}, 'operationTime': Timestamp(1728337087, 29), 'updatedExisting': False}, acknowledged=True)

In [21]:
# 4.3 Write a query to delete all movies that were released before the year 1950.
collection.delete_many({"year": {"$lt": 1950}})

DeleteResult({'n': 640, 'electionId': ObjectId('7fffffff00000000000000a6'), 'opTime': {'ts': Timestamp(1728337093, 525), 't': 166}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1728337093, 534), 'signature': {'hash': b'\xbf>#\x80\x99\xaa\xe7h\xe9\x97\x10\xa0\x92\xd3\x04F\xcb\x99\xc0E', 'keyId': 7376187726187462660}}, 'operationTime': Timestamp(1728337093, 525)}, acknowledged=True)

Exercise 5. Text Search

In [22]:
# 5.1 Ensure the title field is indexed for text search in MongoDB and write a query to search for movies that contain the word "love" in their title.
love_movies = collection.find({"$text": {"$search": "love"}})

In [23]:
# 5.2 Write a text search query to find movies where the word "war" appears in the title or plot, sorted by IMDb rating (Return only the top 5 results).
war_movies = collection.find({"$text": {"$search": "war"}}).sort("imdb.rating", -1).limit(5)

Exercise 6. Combining Multiple Queries (Bonus Exercise)

In [27]:
# find all movies where the genre is "Action" and the IMDb rating is greater than 8 (Sort the results by release year in descending order).
action_high_rated_movies = collection.find({"genres": "Action", "imdb.rating": {"$gt": 8}}).sort("year", -1)

In [26]:
# find 3 movies where the directors contain "Christopher Nolan" and the IMDb rating is greater than 8
nolan_movies = collection.find({"directors": "Christopher Nolan", "imdb.rating": {"$gt": 8}}).sort("imdb.rating", -1).limit(3)